### Exo Lesson 5 (exo_cc.md)

En repartant du dataset "people.csv" (cf. lesson4) dans sa version "clean" finale:
- mettre la colonne `inactive` à `true` pour tous les users dont le `last_seen` date d'au moins un an
- avec une regex: filtrer les numéros de téléphone invalides
- ajouter une colonne indiquant si le numéro de tel correspond à un téléphone portable (06/07)
- ajouter une colonne indiquant si les coordonnées GPS de l'utilisateur correspondent bien au "country"



In [1]:
import requests
import re
import pandas as pd
import matplotlib.pyplot as plt
import datetime

In [2]:
df = pd.read_csv('../../fspot-infmdi-721/lesson4/people.csv', sep=',')

In [3]:
df.head()

,id,first_name,last_name,email address,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,Female,63,$55.18,18.284100,49.632552,0136319724,2019/04/16,False,1.559566e+09,"Palkovice, Czech Republic",entrée/plat/dessert
1,27570,Ruthi,Ross,rross3w@sohu.com,Female,57,$20.37,19.230220,50.466575,NaN,2018/10/23,False,1.567165e+09,"Siewierz, Poland",entrée/plat/dessert
2,27572,Silas,Stourton,silas.stourton3y@answers.com,Male,22,"€32,99",118.831081,24.984813,NaN,2018/12/30,False,1.553692e+09,"Longbo, China",entrée/plat/dessert
3,27435,Roxine,Pettecrew,rpettecrew5@gravatar.com,F,64,"€98,93",121.648987,-8.844744,NaN,2019/03/12,False,1.552349e+09,"Potulando, Indonesia",entrée/plat
4,27558,Margaux,Gowanson,nope@thankyou.,Female,54,$13.30,14.772557,45.160472,NaN,2018/08/13,False,1.543383e+09,"Bribir, Croatia",entrée/plat/dessert


In [4]:
# Convert columns to date type:
df['registration'] = pd.to_datetime(df.registration)
df['last_seen'] = pd.to_datetime(df.last_seen, unit='s')
  
# When missing, last seen should fallback to the registration date:
df['last_seen'] = df.last_seen.combine_first(df.registration)

In [5]:
oneYear = datetime.timedelta(days=365)
df['inactive'] = df['last_seen'] < datetime.datetime.now() - oneYear

In [6]:
df['phone'].str.contains('\s*0\d\.?\s*[\d{2}\.?\s*]{3}\d{2}\s*').value_counts()

True     81
False     2
Name: phone, dtype: int64

In [7]:
df['mobile_phone'] = df['phone'].str.contains('^\s*0[6|7]\.?\s*[\d{2}\s*]{3}\d{2}\s*')

In [45]:
# https://www.latlong.net/c/?lat=0.000000&long=0.000000
def getCountryReverse(lat, lon):
    url = 'https://nominatim.openstreetmap.org/reverse?format=json&lat=%f&lon=%f&addressdetails=1&accept-language=en'%(lat, lon)
    #url = 'https://geocode.xyz/%s,%s?json=1' % (lat, lon)
    #key = '6f9f254c80ab1a'
    #url = 'https://us1.locationiq.com/v1/reverse.php?key=%s&lat=%f&lon=%f&format=json' % (key, lat, lon)
    resp = requests.get(url)
    if resp.status_code == 200:
        json = resp.json()
        return json['address']['country']
       # return json['prov']
    return ''

getCountryReverse(49.632552, 18.284100)

'Czechia'

In [46]:
df['country_reverse'] = df.loc[:,('lat', 'lon')].apply(lambda r: getCountryReverse(r[0], r[1]), axis=1)

In [53]:
df['country_reverse'] = df['country_reverse'].str.replace('PRC', 'China')
df['country_reverse'].value_counts()

China          38
Indonesia      20
Poland         12
Russia         10
Brazil         10
               ..
Afghanistan     1
Panama          1
Yemen           1
Germany         1
Croatia         1
Name: country_reverse, Length: 64, dtype: int64

In [48]:
df['country'] = df['address'].str.extract(',(.*)$')
df['country'] = df['country'].str.replace('\n', ' ').str.strip()

In [54]:
df['country_verified'] = (df.country == df.country_reverse)
df['country_verified'].value_counts()

True     193
False     16
Name: country_verified, dtype: int64

In [50]:
df.head(20)

,id,first_name,last_name,email address,gender,age,money,lon,lat,phone,registration,inactive,last_seen,address,preference,mobile_phone,country_reverse,country,country_verified
0,27625,Leandra,Pabelik,lpabelik5f@yale.edu,Female,63,$55.18,18.284100,49.632552,0136319724,2019-04-16,False,2019-06-03 12:39:57,"Palkovice, Czech Republic",entrée/plat/dessert,False,Czechia,Czech Republic,False
1,27570,Ruthi,Ross,rross3w@sohu.com,Female,57,$20.37,19.230220,50.466575,NaN,2018-10-23,False,2019-08-30 11:41:04,"Siewierz, Poland",entrée/plat/dessert,NaN,Poland,Poland,True
2,27572,Silas,Stourton,silas.stourton3y@answers.com,Male,22,"€32,99",118.831081,24.984813,NaN,2018-12-30,False,2019-03-27 13:10:32,"Longbo, China",entrée/plat/dessert,NaN,PRC,China,False
3,27435,Roxine,Pettecrew,rpettecrew5@gravatar.com,F,64,"€98,93",121.648987,-8.844744,NaN,2019-03-12,False,2019-03-12 00:00:00,"Potulando, Indonesia",entrée/plat,NaN,Indonesia,Indonesia,True
4,27558,Margaux,Gowanson,nope@thankyou.,Female,54,$13.30,14.772557,45.160472,NaN,2018-08-13,False,2018-11-28 05:24:30,"Bribir, Croatia",entrée/plat/dessert,NaN,Croatia,Croatia,True
5,27497,Gordie,Bodicum,gbodicum1v@apache.org,Male,60,$47.26,13.978681,49.786243,0146943857,2017-04-29,False,2018-12-26 07:40:36,"Jince, Czech Republic",entrée/plat/dessert,False,Czechia,Czech Republic,False
6,27539,Dulcine,Stopforth,dstopforth31@forbes.com,Female,22,€12.60,-35.587670,-6.255393,NaN,2017-12-13,False,2019-01-15 15:39:05,"Serrinha, Brazil",entrée/plat/dessert,NaN,Brazil,Brazil,True
7,27469,Denny,Hasnip,dhasnip13@so-net.ne.jp,Female,47,"€75,97",-89.917591,14.113818,NaN,2017-05-01,False,2019-03-08 18:46:46,"Comapa, Guatemala",entrée/plat/dessert,NaN,Guatemala,Guatemala,True
8,27504,Audrie,Huws,NaN,Female,23,$45.50,-71.169359,-14.356896,NaN,2018-04-12,False,2018-12-31 17:39:09,"Maranganí, Peru",entrée/plat/dessert,NaN,Peru,Peru,True
9,27565,Esmeralda,Hollindale,ehollindale3r@guardian.co.uk,Female,29,$60.59,-36.508738,-54.281149,NaN,2017-10-05,True,2018-04-12 15:45:51,"Grytviken, China",entrée/plat/dessert,NaN,South Georgia and South Sandwich Islands,China,False
